VERSION NOTES:
- Fama-MacBeth Statistical Significance tests

# Asset Pricing Tests

In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel.model import FamaMacBeth
from linearmodels.asset_pricing import TradedFactorModel

## Data

In [50]:
ff_data = pd.read_csv('ff_data.csv', index_col=0, parse_dates=True)
monthly_factors = pd.read_csv('monthly_factors.csv', index_col=0, parse_dates=True)
quarterly_factors = pd.read_csv('quarterly_factors.csv', index_col=0, parse_dates=True)

In [51]:
# Monthly
# Test asset clean up
ff_data = ff_data.loc[monthly_factors.index[0]:monthly_factors.index[-1]] / 100

# Factor Models
ff3 = ff_data.iloc[:,0:3]
ff5 = ff_data.iloc[:,0:5]
ff5_mom = pd.concat([ff5, ff_data.iloc[:,31]], axis=1)

# Test Asset Portfolios
size_book_assets_m = ff_data.iloc[:,6:31].apply(lambda x: x - ff_data['RF'])
industry_assets_m = ff_data.iloc[:,32:].apply(lambda x: x - ff_data['RF'])

# Macro Factors
factors_m = pd.concat([monthly_factors, ff_data['Mkt-RF']], axis=1)

In [52]:
# Quarterly
def f(data, last_row=False):
    '''
    Computes the buy and hold for 3 month (1 quarter) return.
    '''
    df = ((1+data).cumprod(axis=0)-1)

    return df.iloc[-1]

ff_data_q = (ff_data).resample('Q').apply(f)

# Factor Models
ff3_q = ff_data_q.iloc[:,0:3]
ff5_q = ff_data_q.iloc[:,0:5]
ff5_mom_q = pd.concat([ff5_q, ff_data_q.iloc[:,31]], axis=1)

# Test Asset Portfolios
size_book_assets_q = ff_data_q.iloc[:,6:31].apply(lambda x: x - ff_data_q['RF'])
industry_assets_q = ff_data_q.iloc[:,32:].apply(lambda x: x - ff_data_q['RF'])

# Macro Factors
factors_q = pd.concat([quarterly_factors, ff_data_q['Mkt-RF']], axis=1).dropna()

## Analysis

Two sets of test assets will be used: 
1) 25 Size-Book Sorted Portfolios 
2) 30 Industry Portfolios

Several factor models will be tested (on each set of test assets):
1) Macro Factors
    - how well do they price the cross-section?
    - what are their risk premia if any?
2) Macro Factors + Market Factor
3) FF3
    - Compare Macro Factors to FF3
4) FF3 + Macro Factors
    - Given the Macro Factors are SMB + HML needed?
5) FF5
6) FF5 + Momentum
    - do the Macro Factors price better than the standard model?
7) FF5 + Momentum + Macro Factors
    - is the standard model subsumed by the Macro Factors?

### Linear Factor Model/SDF Approach

In [61]:
# Renames the factors to make subsequent code cleaner
# _0 denotes MoM or QoQ, _1 denotes YoY or 4QoQ
factors_m.columns = factors_m.columns.str.replace("mom", "0")
factors_m.columns = factors_m.columns.str.replace("yoy", "1")

factors_q.columns = factors_q.columns.str.replace("qoq", "0")
factors_q.columns = factors_q.columns.str.replace("4qoq", "1")

In [69]:
def factor_models(test_assets, factors):
    '''
    Runs all factor models on the given set of test assets.
    Stores the model instances in a dictionary.
    '''
    
    model_results = {}
    
    # Macro Factors
    model1 = TradedFactorModel(test_assets, factors.iloc[:,:2]).fit(cov_type='kernel')
    model_results['macro'] = model1
   
    # Macro Factors + Market
    model2 = TradedFactorModel(test_assets, factors.iloc[:,:]).fit(cov_type='kernel')
    model_results['macro+mkt'] = model2
   
    # FF3
    model3 = TradedFactorModel(test_assets, ff3).fit(cov_type='kernel')
    model_results['ff3'] = model3
    
    # FF3 + Macro Factors
    model4 = TradedFactorModel(test_assets, pd.concat([ff3, factors.iloc[:,:2]], axis=1)).fit(cov_type='kernel')
    model_results['ff3+macro'] = model4
    
    # FF5 + Momentum
    model5 = TradedFactorModel(test_assets, ff5_mom).fit(cov_type='kernel')
    model_results['ff5+mom'] = model5
     
    # FF5 + Momentum + Macro Factors
    model6 = TradedFactorModel(test_assets, pd.concat([ff5_mom, factors.iloc[:,:2]], axis=1)).fit(cov_type='kernel')
    model_results['ff5+macro'] = model6
    
    return model_results

#### Monthly

##### Test Assets: 25 Size-Book Value Sorted Portfolios

In [72]:
size_book_models_mom = factor_models(size_book_assets_m, factors_m[['gdp_0', 'cpi_0', 'Mkt-RF']])
size_book_models_yoy = factor_models(size_book_assets_m, factors_m[['gdp_1', 'cpi_1', 'Mkt-RF']])

In [76]:
# dir(size_book_models_mom['macro+mkt'])
# size_book_models_mom['macro+mkt'].full_summary
# size_book_models_mom['macro+mkt'].betas
# size_book_models_mom['macro+mkt'].alphas

###### MoM Model

In [74]:
size_book_models_mom['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.7276
No. Factors:                          3   J-statistic:                    89.046
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_0          0.0002     0.0001     1.4004     0.1614  -6.241e-05      0.0004
cpi_0         -0.0001  3.603e-05    -2.9049     0.0037     -0.0002  -3.405e-05
Mkt-RF         0.0061     0.0020     3.1136     0.0018      0.0023      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 12
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f50c6d0

GDP's risk premia becomes insignificant (not a surprise from the factor creation stage).

CPI's premia is highly significant, negative and very small.

In [75]:
size_book_models_mom['macro+mkt']._jstat

J-statistic
H0: All alphas are 0
Statistic: 89.0458
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x2ad5f4aed00

In [77]:
12*size_book_models_mom['macro+mkt'].alphas.sort_values()

SMALL LoBM   -0.070035
ME2 BM1      -0.026450
ME3 BM1      -0.022094
BIG LoBM     -0.002565
ME4 BM1       0.000389
ME5 BM4       0.002473
ME5 BM2       0.011611
ME1 BM2       0.012548
ME4 BM2       0.012643
ME1 BM3       0.015558
ME5 BM3       0.015651
ME2 BM2       0.015721
BIG HiBM      0.017111
ME4 BM3       0.018343
ME3 BM3       0.019770
ME3 BM2       0.023827
ME2 BM3       0.024028
ME4 BM4       0.024414
ME3 BM4       0.032213
ME4 BM5       0.033756
ME2 BM4       0.036667
ME2 BM5       0.039072
ME1 BM4       0.044953
ME3 BM5       0.049590
SMALL HiBM    0.053801
Name: alpha, dtype: float64

Reject null that alphas are 0 for the Macro + Mkt Model; the model does not price the portfolios.

The J-statistic itself is the lowest amoung all the models (even FF5+Momentum+Macro). The biggest pricing errors are in the more extreme portfolios (particularly the "SMALL" portfolios).

In [80]:
size_book_models_mom['ff5+mom']._jstat

J-statistic
H0: All alphas are 0
Statistic: 91.4080
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x2ad5f556880

In [82]:
size_book_models_mom['ff5+macro']._jstat

J-statistic
H0: All alphas are 0
Statistic: 80.5827
P-value: 0.0000
Distributed: chi2(25)
WaldTestStatistic, id: 0x2ad5f57d340

Adding macro to FF5 + Momentum enhances pricing (marginally).

In [83]:
size_book_models_mom['ff3+macro']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.9174
No. Factors:                          5   J-statistic:                    83.636
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0061     0.0019     3.1229     0.0018      0.0023      0.0099
SMB            0.0022     0.0012     1.7837     0.0745     -0.0002      0.0046
HML            0.0030     0.0016     1.9047     0.0568  -8.807e-05      0.0062
gdp_0          0.0002     0.0001     1.3668     0.1717  -6.779e-05      0.0004
cpi_0         -0.0001  3.598e-05    -2.9089     0.0036     -0.0002  -3.415e-05
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 9
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f5569d0

Macro factors do not subsume SMB and HML; GDP remains insignificant while CPI is still highly significant.

###### YoY Model

In [92]:
size_book_models_yoy['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 25   R-squared:                      0.7263
No. Factors:                          3   J-statistic:                    87.233
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(25)
Time:                          17:53:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_1          0.0004     0.0002     1.9826     0.0474   4.684e-06      0.0008
cpi_1         -0.0005     0.0002    -2.4412     0.0146     -0.0009  -9.523e-05
Mkt-RF         0.0061     0.0019     3.1261     0.0018      0.0023      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 7
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5f5fb760

##### Test Assets: 30 Industry Portfolios

In [90]:
industry_models_mom = factor_models(industry_assets_m, factors_m[['gdp_0', 'cpi_0', 'Mkt-RF']])
industry__models_yoy = factor_models(industry_assets_m, factors_m[['gdp_1', 'cpi_1', 'Mkt-RF']])

In [91]:
industry_models_mom['macro+mkt']

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 30   R-squared:                      0.5589
No. Factors:                          3   J-statistic:                    70.963
No. Observations:                   595   P-value                         0.0000
Date:                  Sat, Mar 18 2023   Distribution:                 chi2(30)
Time:                          18:02:57                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
gdp_0          0.0002     0.0001     1.3788     0.1680  -6.584e-05      0.0004
cpi_0         -0.0001  3.584e-05    -2.9205     0.0035     -0.0002  -3.443e-05
Mkt-RF         0.0061     0.0019     3.1184     0.0018      0.0023      0.0099
==============================================================================

Covariance estimator:
KernelCovariance, Kernel: bartlett, Bandwidth: 10
See full_summary for complete results
LinearFactorModelResults, id: 0x2ad5e7b5580

### Fama-MacBeth

#### Full-Sample

##### 1st Stage

In [14]:
beta = []
for portfolio in size_book_assets:
    ts = sm.OLS(size_book_assets.loc[size_book_assets.index,portfolio],factors.iloc[:,0:3]).fit()
    beta.append(ts.params[1:])

beta = pd.DataFrame(beta)
beta.index = size_book_assets.columns

##### 2nd Stage

In [16]:
lambdas = []
alphas = []
for t in range(0,size_book_assets.index.shape[0]):
    cs = sm.OLS(np.array(size_book_assets)[t].T, beta).fit()
    lambdas.append(cs.params)
    alphas.append(cs.resid)

In [17]:
lambdas_df = pd.DataFrame(lambdas)
lambdas_df.mean()

gdp_factor    0.019831
cpi_factor   -0.005323
dtype: float64

In [44]:
se_gdp = ((lambdas_df['gdp_factor'] - lambdas_df.mean()['gdp_factor'])**2).sum() / len(lambdas_df)
se_cpi = ((lambdas_df['cpi_factor'] - lambdas_df.mean()['cpi_factor'])**2).sum() / len(lambdas_df)

In [45]:
# Fama-MacBeth t-stat without controlling for autocorrelation 
print(f'{lambdas_df.mean()["gdp_factor"]/se_gdp:0.5f}', f'{lambdas_df.mean()["cpi_factor"]/se_cpi:0.5f}')

0.21410 -4.84238


In [20]:
# Joint test of alphas
alphas_df = pd.DataFrame(alphas)
alpha_hat = alphas_df.mean()

In [58]:
cov_alpha = ((alphas_df - alpha_hat).T @ (alphas_df - alpha_hat)) / len(alphas_df)**2

In [62]:
# Chi Squared Stat
alpha_chi = alpha_hat.T @ np.linalg.inv(cov_alpha) @ alpha_hat
alpha_chi

166.35894853628417

In [63]:
from scipy.stats import chi2

In [64]:
alpha_p_val = 1 - chi2(len(alphas_df)-1).cdf(alpha_chi)
alpha_p_val

1.0

#### 5 year Rolling Window

##### 1st Stage

In [30]:
from statsmodels.regression.rolling import RollingOLS

In [31]:
beta = {}
for portfolio in size_book_assets:
    ts = RollingOLS(size_book_assets.loc[size_book_assets.index,portfolio],factors.iloc[:,0:3], window=60).fit()
    beta[portfolio] = ts.params[1:]

In [32]:
betas_T = {}
for t in range(60,len(size_book_assets)-60):
    betas = []
    for portfolio in size_book_assets.columns:
        betas.append(beta[portfolio].iloc[t])
    
    betas_df = pd.DataFrame(np.array(betas), index=size_book_assets.columns, columns=['const', 'gdp_factor', 'cpi_factor'])
    
    betas_T[t] = betas_df

##### 2nd Stage

In [33]:
lambdas = []
alphas = []
for t in range(60,len(size_book_assets)-60):
    cs = sm.OLS(np.array(size_book_assets)[t].T, betas_T[t]).fit()
    lambdas.append(cs.params)
    alphas.append(cs.resid)

In [34]:
pd.DataFrame(lambdas).mean()

const         0.901521
gdp_factor    0.000045
cpi_factor   -0.001267
dtype: float64

In [401]:
# pd.DataFrame(alphas).mean()

### GMM?